In [1]:
import os
import sys
projectDir = '/home/thibaud/repos/anaconda/rakuten'
if projectDir not in sys.path:
    sys.path.append(projectDir)

import multiprocessing
num_cores = multiprocessing.cpu_count()

#Define path in the config file
import src.config as config
config.path_to_project = projectDir #directory of the project
config.path_to_data = os.path.join(projectDir, 'data', 'clean') #path to the data (dataframe)
config.path_to_results = os.path.join(projectDir, 'results') #path to where the summary of the benchmark results will be saved (csv)
config.path_to_images = os.path.join(projectDir, 'data', 'images', 'image_train_resized')  #path to the folder containing images
config.path_to_models = os.path.join(projectDir, 'results') #path to the folder where the models will be saved


import pandas as pd
import numpy as np
import tensorflow as tf

from src.text.classifiers import TFbertClassifier
from src.image.classifiers import ImgClassifier
from src.multimodal.classifiers import TFmultiClassifier

from src.utils.batch import fit_save_all
from src.utils.plot import plot_training_history

import datetime

2024-03-11 17:55:59.817377: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-11 17:55:59.838424: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-11 17:56:00.964047: I itex/core/wrapper/itex_gpu_wrapper.cc:35] Intel Extension for Tensorflow* GPU backend is loaded.
2024-03-11 17:56:00.988030: I itex/core/wrapper/itex_cpu_wrapper.cc:52] Intel Extension for Tensorflow* AVX2 CPU backend is loaded.
2024-03-11 17:56:01.027379: W itex/core/ops/op_init.cc:58] Op: _QuantizedMaxPool3D is already registere

In [2]:
def Rakuten_img_path(img_folder, imageid, productid, suffix=''):
    """ retrurns the path to the image of a given productid and imageid"""

    df = pd.DataFrame(pd.concat([imageid, productid], axis=1))

    img_path = df.apply(lambda row:
                        os.path.join(img_folder, 'image_'
                                     + str(row['imageid'])
                                     + '_product_'
                                     + str(row['productid'])
                                     + suffix
                                     + '.jpg'),
                        axis=1)

    return img_path

In [3]:
data_train = pd.read_csv(os.path.join(config.path_to_data, 'df_train_index.csv'))
data_train['testset'] = False
data_test = pd.read_csv(os.path.join(config.path_to_data, 'df_test_index.csv'))
data_test['testset'] = True
data = pd.concat([data_train, data_test], axis=0)

#merging text into token column
colnames = ['designation_translated', 'description_translated'] #['designation', 'description']#
data['tokens'] = data[colnames].apply(lambda row: ' '.join(s.lower() for s in row if isinstance(s, str)), axis=1)

#path to images into img_path column
data['img_path'] = Rakuten_img_path(img_folder=config.path_to_images,
                             imageid=data['imageid'], productid=data['productid'], suffix='_resized')

In [4]:
#labels of encoded classes
class_labels = data.groupby('prdtypedesignation')['prdtypeindex'].first().reset_index()
class_labels.index = class_labels['prdtypeindex']
class_labels = class_labels.drop(columns='prdtypeindex').sort_index()

## Creating train and test sets

In [5]:
Img_train = data.loc[~data['testset'], 'img_path']
Img_test = data.loc[data['testset'], 'img_path']

Txt_train = data.loc[~data['testset'], 'tokens']
Txt_test = data.loc[data['testset'], 'tokens']

y_train = data.loc[~data['testset'],'prdtypeindex']
y_test = data.loc[data['testset'],'prdtypeindex']

#To be fed into any of our sklearn classifiers, X_train and X_test
#should be dataframes with columns tokens and img_path
X_train = pd.DataFrame({'tokens': Txt_train, 'img_path': Img_train})
X_test = pd.DataFrame({'tokens': Txt_test, 'img_path': Img_test})

#All data for cross-validated scores
X = pd.concat([X_train, X_test], axis=0)
y = pd.concat([y_train, y_test], axis=0)

#Number of classes
num_classes = len(np.unique(data['prdtypeindex']))

In [7]:
# model = MLClassifier(base_name='LogisticRegression', vec_method='tfidf')
# model = model.load('LogisticRegression_tfidf')
# model.predict(X_test)

# CBOW benchmark

In [16]:
import src.text.cbow_vectorizer as cbow
import src.utils.plot as uplot
from sklearn.experimental import enable_halving_search_cv
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, HalvingGridSearchCV
from sklearn.pipeline import Pipeline


vcbow = cbow.CBowVectorizer()
model = LogisticRegression()
pipeline = Pipeline([
    ('cbow', vcbow),
    ('logreg', model)
])
# vector_size=500, window=10, min_count=2, workers=4
param_grid = {
    'cbow__vector_size': [100],
    'cbow__window': [10],
    'cbow__min_count': [2, 3, 4],
    'cbow__workers': [14],
    'logreg__C': [0.1, 1, 10],
    'logreg__max_iter': [2000]
}


grid = HalvingGridSearchCV(pipeline, param_grid, cv=3, scoring='accuracy', n_jobs=14)

grid.fit(X_train['tokens'], y_train)

best_params = grid.best_params_
best_score = grid.best_score_

print("Meilleurs hyperparamètres :", best_params)
print("Meilleur score :", best_score)



# Meilleurs hyperparamètres : {'cbow__min_count': 2, 'cbow__vector_size': 300, 'cbow__window': 10, 'cbow__workers': 14, 'logreg__C': 0.1, 'logreg__max_iter': 2000}
# Meilleur score : 0.6876582464817759

# Meilleurs hyperparamètres : {'cbow__min_count': 3, 'cbow__vector_size': 100, 'cbow__window': 10, 'cbow__workers': 14, 'logreg__C': 10, 'logreg__max_iter': 2000}
# Meilleur score : 0.6853323912147441


Meilleurs hyperparamètres : {'cbow__min_count': 3, 'cbow__vector_size': 100, 'cbow__window': 10, 'cbow__workers': 14, 'logreg__C': 10, 'logreg__max_iter': 2000}
Meilleur score : 0.6853323912147441


In [ ]:
y_pred = grid.predict(X_test)

uplot.classification_results(y_test, y_pred, class_labels, 'LogisticRegression_cbow')

## Dummy classifier

In [10]:
#Dummy classifier on tfidf
# dum_classifier = MLClassifier(base_name='dummyclassifier')
# dum_classifier.fit(X_train, y_train);
# dum_classifier.classification_score(X_test, y_test)
# cv_scores = dum_classifier.cross_validate(X, y, cv=10)
# dum_classifier.save('text/dummy')

## Bag of word based benchmarks

In [6]:
params_list = []
class_type = 'MLClassifier'

#grid search number of folds
nfolds_grid = 0

#cross-validation of f1-score
nfolds_cv = 0

#Bag of word parameters
vec_method = 'tfidf'

params_list.append({'modality': 'text',
                    'class': class_type,
                    'base_name': 'LogisticRegression', 
                    'vec_method': vec_method, 
                    'param_grid': {'C': [0.1, 1, 2], 'penalty': ['l2'], 'max_iter': [2000]},
                    'nfolds_grid': 3, 'nfolds_cv': nfolds_cv
                   })
# params_list.append({'modality': 'text',
#                     'class': class_type,
#                     'base_name': 'MultinomialNB', 
#                     'vec_method': vec_method, 
#                     'param_grid': {'alpha': [0.02], 'fit_prior': [True]},
#                     'nfolds_grid': nfolds_grid, 'nfolds_cv': nfolds_cv
# #                     })
# params_list.append({'modality': 'text',
#                     'class': class_type,
#                     'base_name': 'RandomForestClassifier', 
#                     'vec_method': vec_method, 
#                     'param_grid': {'n_estimators': [200], 'max_depth': [500,1000]},
#                     'nfolds_grid': 0, 'nfolds_cv': 0
#                     })
# params_list.append({'modality': 'text',
#                     'class': class_type,
#                     'base_name': 'xgboost', 
#                     'vec_method': vec_method, 
#                     'param_grid': {'n_estimators': [200], 'objective': ['multi:softprob'], 'max_depth':[6], 'reg_alpha':[0]},
#                     'nfolds_grid': nfolds_grid, 'nfolds_cv': nfolds_cv
#                     })
# params_list.append({'modality': 'text',
#                     'class': class_type,
#                     'base_name': 'LinearSVC', 
#                     'vec_method': vec_method, 
#                     'param_grid': {'C': np.arange(0.5, 1.5, 0.1), 'penalty': ['l2']},
#                     'nfolds_grid': 5, 'nfolds_cv': 10
#                     })

results = fit_save_all(params_list, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, result_file_name = 'results_benchmark_text.csv')


Fitting:  LogisticRegression tfidf
{'C': 0.1, 'penalty': 'l2', 'max_iter': 2000}
GridSearch:  {'C': 0.1, 'max_iter': 2000, 'penalty': 'l2'}
Test set, f1score:  0.6817626342598585


In [ ]:
# 3min57 Halving
# Fitting:  LogisticRegression tfidf
# {'C': 0.1, 'penalty': 'l2', 'max_iter': 2000}
# GridSearch:  {'C': 2, 'max_iter': 2000, 'penalty': 'l2'}
# Test set, f1score:  0.6817626342598585

Fitting:  LogisticRegression tfidf
{'C': 0.1, 'penalty': 'l2', 'max_iter': 2000}
GridSearch:  {'C': 0.1, 'max_iter': 2000, 'penalty': 'l2'}
Test set, f1score:  0.6817626342598585

## Fetch and check the saved result file

In [6]:
results = pd.read_csv(os.path.join(config.path_to_results,'results_benchmark_text.csv'), index_col=0)
results.head()

,modality,class,vectorization,classifier,tested_params,best_params,score_test,score_cv_test,score_cv_train,fit_cv_time,model_path
0,text,MLClassifier,tfidf,LinearSVC,"{'C': array([0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1...","{'C': 0.5, 'penalty': 'l2'}",0.82398,[0.82403265 0.82531629 0.82968601 0.82028095 0...,[0.96525381 0.96556959 0.96546323 0.96552612 0...,[8.3740375 8.30071878 8.0570724 8.07186079 8...,text/LinearSVC_tfidf


## Word2vec based benchmarks

In [ ]:
params_list = []
class_type = 'MLClassifier'

#grid search number of folds
nfolds_grid = 0

#cross-validation of f1-score
nfolds_cv = 0

#Word2vec parameters
vec_method = 'skipgram'
vector_size = 500

#List of parameters to batch over
params_list.append({'modality': 'text',
                    'class': class_type, 
                    'base_name': 'LogisticRegression',
                    'vec_method': vec_method,
                    'param_grid': {'C': [10], 'penalty': 'l2',
                                   'vec_params':[{'workers': num_cores-1, 'vector_size': vector_size}]},
                    'nfolds_grid': nfolds_grid, 'nfolds_cv': nfolds_cv                    
                    })
params_list.append({'modality': 'text',
                    'class': class_type,
                    'base_name': 'RandomForestClassifier', 
                    'vec_method': vec_method, 
                    'param_grid': {'n_estimators': [200], 'max_depth': [500],
                                   'vec_params':[{'workers': num_cores-1, 'vector_size': vector_size}]},
                    'nfolds_grid': nfolds_grid, 'nfolds_cv': nfolds_cv
                    })
params_list.append({'modality': 'text',
                    'class': class_type,
                    'base_name': 'xgboost', 
                    'vec_method': vec_method, 
                    'param_grid': {'n_estimators': [200], 'objective': ['multi:softprob'], 'max_depth':[6], 'reg_alpha':[0],
                                   'vec_params':[{'workers': num_cores-1, 'vector_size': vector_size}]},
                    'nfolds_grid': nfolds_grid, 'nfolds_cv': nfolds_cv
                    })
params_list.append({'modality': 'text',
                    'class': class_type, 
                    'base_name': 'SVC',
                    'vec_method': vec_method,
                    'param_grid': {'C': [10], 'kernel': ['rbf'],
                                   'vec_params':[{'workers': num_cores-1, 'vector_size': vector_size}]},
                    'nfolds_grid': nfolds_grid, 'nfolds_cv': nfolds_cv
                    })

results = fit_save_all(params_list, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, result_file_name = 'results_benchmark_text.csv')

## Fetch and check the saved result file

In [ ]:
## Fetch and check the saved result file
results = pd.read_csv(os.path.join(config.path_to_results,'results_benchmark_text.csv'), index_col=0)
results.head()

## Example usage of ML classifier

In [70]:
#SVC on tfidf
svc_w2v_classifier = MLClassifier(base_name='SVC', C=10, kernel='rbf', vec_method = 'skipgram', vec_params={'workers': num_cores-1, 'vector_size': 512})
svc_w2v_classifier.fit(X_train, y_train);
svc_w2v_classifier.classification_score(X_test, y_test)
svc_w2v_classifier.cross_validate(X, y, cv=10)
svc_w2v_classifier.save('text/SVC_skipgram')

0.7695062608525401